<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 4*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Apply regularization techniques to your model. 

*Don't forgot to switch to GPU on Colab!*

## Regularization

Using your best performing model from the previous module, apply each of the following regularization strategies: 
* Early Stopping
* Dropout
* Weight Decay
* Weight Constraint


In [5]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

def load_quickdraw10(path):
  data = np.load(path)
  X = data['arr_0']
  y = data['arr_1']

  print(X.shape)
  print(y.shape)

  X, y = shuffle(X, y)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  
  return X_train, y_train, X_test, y_test

C:\ProgramData\Anaconda3\envs\U4-S2-NN\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.0) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [6]:
X_train, y_train, X_test, y_test = load_quickdraw10('quickdraw10.npz')

(100000, 784)
(100000,)


In [7]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import ReLU, LeakyReLU
import tensorflow as tf
import os

###  Early stopping

logdir = os.path.join("logs", "EarlyStopping-Loss")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_accuracy', min_delta=0.005, patience=5)

model = tf.keras.Sequential([
    Flatten(input_shape=(28,28)),
    Dense(128, activation='relu'),
    ReLU(negative_slope=.01),
    Dense(128),
    ReLU(negative_slope=.01),
    Dense(128),
    ReLU(negative_slope=.01),
    Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

model.fit(X_train, y_train, 
          epochs=100, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/100
   1/2500 [..............................] - ETA: 0s - loss: 125.6458 - accuracy: 0.0625WARNING:tensorflow:From C:\ProgramData\Anaconda3\envs\U4-S2-NN\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
2500/2500 [==============================] - 3s 1ms/step - loss: 1.5268 - accuracy: 0.6972 - val_loss: 0.7229 - val_accuracy: 0.7836
Epoch 2/100
2500/2500 [==============================] - 2s 943us/step - loss: 0.6592 - accuracy: 0.8008 - val_loss: 0.6649 - val_accuracy: 0.8047
Epoch 3/100
2500/2500 [==============================] - 2s 920us/step - loss: 0.5810 - accuracy: 0.8237 - val_loss: 0.5938 - val_accuracy: 0.8209
Epoch 4/100
2500/2500 [==============================] - 2s 871us/step - loss: 0.5337 - accuracy: 0.8386 - val_loss: 0.5708 - val_accuracy: 0.8320
Epoch 5/100
2500/2500 [=======

In [12]:
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.layers import Dropout

## DROPOUT

logdir = os.path.join("logs", "Dropout-MaxNorm")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_accuracy', min_delta=0.005, patience=5)

# Normal values tend to be between 0.2-0.5
drop = 0.2
model = tf.keras.Sequential([
    Flatten(input_shape=(28,28)),
    Dense(128, activation='relu', kernel_constraint=MaxNorm(3)),
    Dropout(drop),
    Dense(128, activation='relu', kernel_constraint=MaxNorm(3)),
    Dropout(drop),
    Dense(128, activation='relu', kernel_constraint=MaxNorm(3)),
    Dropout(drop),
    Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='nadam', 
              metrics=['accuracy'])

model.fit(X_train, y_train, 
          epochs=100, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/100
2500/2500 [==============================] - 3s 1ms/step - loss: 2.2696 - accuracy: 0.5206 - val_loss: 0.9707 - val_accuracy: 0.7163
Epoch 2/100
2500/2500 [==============================] - 3s 1ms/step - loss: 1.0702 - accuracy: 0.6747 - val_loss: 0.8439 - val_accuracy: 0.7487
Epoch 3/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.9637 - accuracy: 0.7084 - val_loss: 0.7859 - val_accuracy: 0.7727
Epoch 4/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.9037 - accuracy: 0.7266 - val_loss: 0.7502 - val_accuracy: 0.7821
Epoch 5/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.8586 - accuracy: 0.7394 - val_loss: 0.7037 - val_accuracy: 0.7933
Epoch 6/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.8203 - accuracy: 0.7497 - val_loss: 0.7139 - val_accuracy: 0.7925
Epoch 7/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.7980 - accuracy: 0.7612 - val_loss: 0.6701 - val_ac

In [13]:
###Weight constraint

logdir = os.path.join("logs", "WC-MaxNorm")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_accuracy', min_delta=0.005, patience=5)

# Normal values tend to be 3-4
wc = 3
model = tf.keras.Sequential([
    Flatten(input_shape=(28,28)),
    Dense(128, activation='relu', kernel_constraint=MaxNorm(3)),
    Dense(128, activation='relu', kernel_constraint=MaxNorm(3)),
    Dense(128, activation='relu', kernel_constraint=MaxNorm(3)),
    Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='nadam', 
              metrics=['accuracy'])

model.fit(X_train, y_train, 
          epochs=100, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/100
2500/2500 [==============================] - 3s 1ms/step - loss: 1.4891 - accuracy: 0.6987 - val_loss: 0.7113 - val_accuracy: 0.7850
Epoch 2/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6440 - accuracy: 0.8047 - val_loss: 0.6096 - val_accuracy: 0.8187
Epoch 3/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.5630 - accuracy: 0.8283 - val_loss: 0.5524 - val_accuracy: 0.8366
Epoch 4/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.5152 - accuracy: 0.8437 - val_loss: 0.5370 - val_accuracy: 0.8451
Epoch 5/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.4818 - accuracy: 0.8548 - val_loss: 0.5358 - val_accuracy: 0.8406
Epoch 6/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.4522 - accuracy: 0.8639 - val_loss: 0.5305 - val_accuracy: 0.8482
Epoch 7/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.4370 - accuracy: 0.8689 - val_loss: 0.5285 - val_ac

In [14]:
from tensorflow.keras import regularizers

#weight decay

logdir = os.path.join("logs", "L2-weight decay")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_accuracy', min_delta=0.005, patience=5)

# Normal values tend to be 0 to 0.0001 on log scale
wd = 0.001
model = tf.keras.Sequential([
    Flatten(input_shape=(28,28)),
    Dense(128, activation='relu', kernel_regularizer=regularizers.L2(wd)),
    Dense(128, activation='relu', kernel_regularizer=regularizers.L2(wd)),
    Dense(128, activation='relu', kernel_regularizer=regularizers.L2(wd)),
    Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='nadam', 
              metrics=['accuracy'])

model.fit(X_train, y_train, 
          epochs=100, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/100
2500/2500 [==============================] - 3s 1ms/step - loss: 1.8990 - accuracy: 0.7045 - val_loss: 1.0417 - val_accuracy: 0.7888
Epoch 2/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.9450 - accuracy: 0.8008 - val_loss: 0.8566 - val_accuracy: 0.8141
Epoch 3/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.7596 - accuracy: 0.8263 - val_loss: 0.7242 - val_accuracy: 0.8292
Epoch 4/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6567 - accuracy: 0.8386 - val_loss: 0.6782 - val_accuracy: 0.8336
Epoch 5/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6179 - accuracy: 0.8470 - val_loss: 0.6302 - val_accuracy: 0.8454
Epoch 6/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.5986 - accuracy: 0.8519 - val_loss: 0.6402 - val_accuracy: 0.8439
Epoch 7/100
2500/2500 [==============================] - 3s 1ms/step - loss: 0.5849 - accuracy: 0.8563 - val_loss: 0.6324 - val_ac

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

## Deploy

Save your model's weights using the Checkpoint function. Try reloading the model and making inference on your validation dataset.

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint

stop = EarlyStopping(monitor='val_accuracy', min_delta=0.005, patience=5)
mcp = ModelCheckpoint('best_weights.h5', 
                      monitor='val_accuracy', 
                      verbose=1, 
                      save_best_only=True,
                      save_weights_only=True)

def get_model():
    model = tf.keras.Sequential([
        Flatten(input_shape=(28,28)),
        Dense(128, activation='relu'),
        ReLU(negative_slope=.01),
        Dense(128),
        ReLU(negative_slope=.01),
        Dense(128),
        ReLU(negative_slope=.01),
        Dense(10, activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

    return model

model = get_model()

model.fit(X_train, y_train, 
          epochs=100, 
          validation_data=(X_test,y_test),
          callbacks=[stop, mcp])

Epoch 1/100
2484/2500 [============================>.] - ETA: 0s - loss: 1.4770 - accuracy: 0.7105WARNING:tensorflow:Model was constructed with shape (None, 28, 28) for input Tensor("flatten_6_input:0", shape=(None, 28, 28), dtype=float32), but it was called on an input with incompatible shape (32, 784).

Epoch 00001: val_accuracy improved from -inf to 0.78765, saving model to best_weights.h5
2500/2500 [==============================] - 3s 1ms/step - loss: 1.4719 - accuracy: 0.7109 - val_loss: 0.6828 - val_accuracy: 0.7876
Epoch 2/100
2471/2500 [============================>.] - ETA: 0s - loss: 0.6284 - accuracy: 0.8074
Epoch 00002: val_accuracy improved from 0.78765 to 0.81550, saving model to best_weights.h5
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6279 - accuracy: 0.8076 - val_loss: 0.6067 - val_accuracy: 0.8155
Epoch 3/100
2490/2500 [============================>.] - ETA: 0s - loss: 0.5579 - accuracy: 0.8309
Epoch 00003: val_accuracy improved from 0.81550 

In [19]:
model.load_weights('best_weights.h5')
model.evaluate(X_test, y_test)

625/625 [==============================] - 0s 394us/step - loss: 0.5039 - accuracy: 0.8594


[0.5039432644844055, 0.8593500256538391]

In [20]:
model2 = get_model()
model2.load_weights('best_weights.h5')
model2.evaluate(X_test, y_test)

625/625 [==============================] - 0s 697us/step - loss: 0.5039 - accuracy: 0.8594


[0.5039432644844055, 0.8593500256538391]

### Stretch Goals
- Mount your Google Drive to Colab to persist your model checkpoint files. 
- Research L2 normalization (weight decay)
- Write a custom callback function to stop training after you reach .88 validation accuracy. 
- Select a new dataset and apply a neural network to it.
- Research TensorFlow Serving
- Play [QuickDraw](https://quickdraw.withgoogle.com/data)
- Create a static webpage using TensorFlow.js to serve a model. Check out [Teachable Machine Learning](https://teachablemachine.withgoogle.com/) for ideas. 